In [1]:
using Pkg 
Pkg.instantiate()
using qAlgebra

using BenchmarkTools

Precompiling project...
   1012.9 ms  ✓ qAlgebra
  1 dependency successfully precompiled in 1 seconds. 7 already precompiled.


In [2]:
qspace = StateSpace("alpha", "beta(t)", "gamma_i", "delta_i", operators=["A(!i)", "B(U,H,i)"], h=QubitPM("eta"), i=(3, QubitPauli("sigma")), b=Ladder())

StateSpace: [β(t), γᵢ, γⱼ, γₖ, δᵢ, δⱼ, δₖ, α]
   - SubSpace ["h"]: PM Qubit (Fermionic):  ηᵖₚ, ηᵐₚ, ηᶻₚ, ηᴵₚ (identity)
   - SubSpace ["i", "j", "k"]: Pauli Qubit (Fermionic):  σˣₚ, σʸₚ, σᶻₚ, σᴵₚ (identity)
   - SubSpace ["b"]: Ladder (Bosonic):  p†, p
   - Op: A
   - Op: B(H,U)


In [3]:
var_dict, op_dict, abstract_dict = base_operators(qspace)
alpha = base_operators(qspace, "alpha")
beta = base_operators(qspace, "beta")
gamma_i, gamma_j, gamma_k = base_operators(qspace, "gamma", do_dict=false)
ph, mh, zh = base_operators(qspace, "h", do_dict=false)
xi,yi,zi, pi, mi = base_operators(qspace, "i", do_dict=false)
xj, yj, zj, pj, mj = base_operators(qspace, "j", do_dict=false)
xk, yk, zk, pk, mk = base_operators(qspace, "k", do_dict=false)
b, n = base_operators(qspace, "b", do_dict=false)
I = base_operators(qspace, "I")
A = base_operators(qspace, "A")
println("Done")

Done


In [5]:
A = base_operators(qspace, "A", do_fun=true)
A1 = base_operators(qspace, "A_1", do_fun=false)
A = A()

A

In [5]:
Sum("j", alpha * yi * yj + Sum("k", beta * alpha^2 * xi * xi * xk))

∑ⱼ⁼(ασʸᵢσʸⱼ+∑ₖ⁼β(t)α²σˣₖ)

In [6]:
@benchmark Sum("j", alpha * yi * yj + Sum("k", beta * alpha^2 * xi * xj * xk))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  56.041 μs …  12.241 ms  ┊ GC (min … max): 0.00% … 98.88%
 Time  (median):     57.875 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   61.892 μs ± 170.891 μs  ┊ GC (mean ± σ):  4.63% ±  1.71%

      ▄██▃▁                                                     
  ▂▃▄▆█████▆▆▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂ ▃
  56 μs           Histogram: frequency by time         70.4 μs <

 Memory estimate: 51.72 KiB, allocs estimate: 831.

In [7]:
expr = neq(log(Sum("j", alpha * yi * yj + Sum("k", beta * gamma_j * gamma_k * xi * xj * xk))))

log(α+2β(t)γᵢ²σˣᵢ+∑ⱼ(ασʸᵢσʸⱼ+β(t)γⱼ²σˣᵢ+β(t)γᵢγⱼσˣⱼ)+∑ₖβ(t)γᵢγₖσˣₖ+∑₍ⱼₖ₎β(t)γⱼγₖσˣᵢσˣⱼσˣₖ)

In [8]:
d_dt(xi, expr)

d(σˣᵢ) / dt = log(α+2β(t)γᵢ²σˣᵢ+∑ⱼ(ασʸᵢσʸⱼ+β(t)γⱼ²σˣᵢ+β(t)γᵢγⱼσˣⱼ)+∑ₖβ(t)γᵢγₖσˣₖ+∑₍ⱼₖ₎β(t)γⱼγₖσˣᵢσˣⱼσˣₖ)

In [9]:
function extract_qabstract(q::qExpr)::qAbstract
    if length(q) > 1 
        error("Cannot substitute composites. abstract_op must contain only an abstract operator.")
    end
    term = q.terms[1]
    if !isa(term, qAtomProduct)
        error("qExpr must contain only a qAtomProduct")
    end
    return extract_qabstract(term)
end
function extract_qabstract(term::qAtomProduct)::qAbstract
    if length(term.expr) != 1 || !isa(term.expr[1], qAbstract)
        error("abstract_op must contain exactly one qAbstract")
    end
    return term.expr[1]
end
#now same for qAtom 
function extract_qatom(q::qExpr)::qAtom
    if length(q) > 1 
        error("Cannot substitute composites. abstract_op must contain only an abstract operator.")
    end
    term = q.terms[1]
    if !isa(term, qAtomProduct)
        error("qExpr must contain only a qAtomProduct")
    end
    return extract_qatom(term)
end
function extract_qatom(term::qAtomProduct)::qAtom
    if length(term.expr) != 1 || !isa(term.expr[1], qAtom)
        error("abstract_op must contain exactly one qAbstract")
    end
    return term.expr[1]
end

extract_qatom (generic function with 2 methods)

In [10]:
using ComplexRationals
function term_equal_indexes_ss(term::qTerm, index1::Int, index2::Int, statespace::StateSpace)::Tuple{Bool, Vector{qTerm}, Vector{ComplexRational}}
    op1 = term.op_indices[index1]
    op2 = term.op_indices[index2]
    neutral = subspace.op_set.neutral_element
    if op1 === neutral && op2 === neutral
        return false, qTerm[term], ComplexRational[ComplexRational(1,0,1)]
    end
    subspace_ind = statespace.subspace_by_ind[index1]
    subspace = statespace.subspaces[subspace_ind]
    results = subspace.op_set.op_product(op1, op2)
    new_terms = qTerm[]
    new_coeffs = ComplexRational[]
    for (coeff, op) in results
        new_term = copy(term)
        new_term.op_indices[index2] = op
        new_term.op_indices[index1] = neutral
        push!(new_terms, new_term)
        push!(new_coeffs, coeff)
    end
    return true, new_terms, new_coeffs
end 

term_equal_indexes_ss (generic function with 1 method)

In [22]:
# substitute abstract operator 
# input is abstract_op, replacement and target
simpleQ = Union{qExpr, qAtomProduct}
# Deals with index_map mapping one index to another ithin qAbstract
function qAtom_index_flip(q::qAtom, index_map::Vector{Tuple{Int,Int}}, statespace::StateSpace)::Vector{qAtomProduct}
    qs::Vector{qAtom} = [q]
    cs::Vector{ComplexRational} = [ComplexRational(1,0,1)]
    for (index1, index2) in index_map
        new_qs::Vector{qAtom} = []    
        new_cs::Vector{ComplexRational} = []
        for (qi, ci) in zip(qs, cs)
            _, new_terms, new_coeffs = term_equal_indexes_ss(qi, index1, index2, statespace)
            append!(new_qs, new_terms)
            append!(new_cs, new_coeffs*ci)
        end
        qs = new_qs
        cs = new_cs
    end
    return [qAtomProduct(statespace, statespace.fone*c, [q]) for (q, c) in zip(qs, cs)]
end
function substitute_qAtom(abstract_op::qAbstract, replacement::qAtom, target::qTerm, statespace::StateSpace)::Vector{qAtomProduct}
    return [qAtomProduct(statespace, statespace.fone, [target])]
end
function substitute_qAtom(abstract_op::qAbstract, replacement::qAtom, target::qAbstract, statespace::StateSpace)::Vector{qAtomProduct}
    # check if its the same qAbstract operator 
    if target.key_index == abstract_op.key_index && target.sub_index == abstract_op.sub_index 
        qs = qAtom_index_flip(replacement, target.index_map, statespace)

        if target.exponent != 1
            qs = qs.^target.exponent
        end
        if target.dag
            qs = qs'
        end
        return qs
    else
        return [qAtomProduct(statespace, statespace.fone, [target])]
    end
end

function substitute(abstract_op::qAbstract, replacement::qAtom, target::qAtomProduct)::Vector{qComposite}
    # recursively navigate expression, and substitue
    statespace = target.statespace
    expr = target.expr
    coeff_fun = target.coeff_fun
    new_expr::qExpr = qExpr(target.statespace, substitute_qAtom(abstract_op, replacement, expr[1], statespace))
    for t in expr[2:end]
        new_terms = substitute_qAtom(abstract_op, replacement, t, statespace)
        new_new_expr = new_expr * new_terms[1]
        for t in new_terms[2:end]
            new_new_expr += new_expr + t
        end
        new_expr = new_new_expr
    end
    terms = simplify(new_expr).terms
    return [qAtomProduct(statespace, coeff_fun*t.coeff_fun, t.expr) for t in terms]  
end


function substitute(abstract_op::Union{simpleQ, qAbstract}, replacement::Union{simpleQ, qAtom}, target::qExpr)::qExpr
    if !isa(abstract_op, qAbstract)
        abstract_op = extract_qabstract(abstract_op)
    end
    if !isa(replacement, qAtom)
        replacement = extract_qatom(replacement)
    end
    return substitute(abstract_op, replacement, target)
end
function substitute(abstract_op::qAbstract, replacement::qAtom, target::qExpr)::qExpr
    # recursively navigate expression, and substitue
    new_terms = qComposite[]
    for term in target.terms
        append!(new_terms, substitute(abstract_op, replacement, term))
    end
    return qExpr(target.statespace, new_terms)
end
function substitute(a::qAbstract, r::qAtom, targ::T) where T<:qComposite
    # T<:qMultiComposite is *also* <:qComposite, 
    # so we need the qMultiComposite method to be more specific
    cp = copy(targ)
    cp.expr = substitute(a, r, targ.expr)
    return [cp]  # Tuple or Vector, depending on your convention
end

# For anything that holds *many* sub‑expressions
function substitute(a::qAbstract, r::qAtom, targ::T) where T<:qMultiComposite
    cp = copy(targ)
    cp.expr = map(x -> substitute(a, r, x), targ.expr)
    return [cp]
end

function substitute(abstract_op::Union{simpleQ, qAbstract}, replacement::Union{simpleQ, qAtom}, target::diff_qEQ)::diff_qEQ 
    if !isa(abstract_op, qAbstract)
        abstract_op = extract_qabstract(abstract_op)
    end
    if !isa(replacement, qAtom)
        replacement = extract_qatom(replacement)
    end
    return substitute(abstract_op, replacement, target)
end
function substitute(abstract_op::qAbstract, replacement::qAtom, target::diff_qEQ)::diff_qEQ
    lhs = substitute(abstract_op, replacement, target.left_hand_side)
    if length(lhs) != 1
        error("Substitution of $abstract_op with $replacement in $target did not result in a single term.")
    end
    lhs = lhs[1]
    rhs = substitute(abstract_op, replacement, target.expr)
    return diff_qEQ(target.statespace, lhs, rhs, target.braket)
end

substitute (generic function with 7 methods)

In [9]:
expr = neq(log(Sum("j", alpha * A * yj + Sum("k", beta * gamma_j * gamma_k * A * xj * xk))))
dA_dt = d_dt(A, expr)
dx_dt = substitute(A, xi, dA_dt)

d(σˣᵢ) / dt = log(-iασᶻᵢ+2β(t)γᵢ²σˣᵢ+∑ⱼ(ασˣᵢσʸⱼ+β(t)γⱼ²σˣᵢ+β(t)γᵢγⱼσˣⱼ)+∑ₖβ(t)γᵢγₖσˣₖ+∑₍ⱼₖ₎β(t)γⱼγₖσˣᵢσˣⱼσˣₖ)

In [10]:
expr = neq(log(Sum("j", alpha * A * yj + Sum("k", beta * gamma_j * gamma_k * A * xj * xk))))

log(ασʸᵢA+2β(t)γᵢ²σˣᵢAσˣᵢ+∑ⱼ(ασʸⱼA+β(t)γⱼ²σˣⱼAσˣⱼ+β(t)γᵢγⱼσˣⱼAσˣᵢ)+∑ₖβ(t)γᵢγₖσˣᵢAσˣₖ+∑₍ⱼₖ₎β(t)γⱼγₖσˣⱼAσˣₖ)

In [10]:
@benchmark substitute(A, xi, dA_dt)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  230.501 μs …  11.884 ms  ┊ GC (min … max): 0.00% … 96.98%
 Time  (median):     235.501 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   248.854 μs ± 236.525 μs  ┊ GC (mean ± σ):  4.14% ±  4.77%

      ▁▅█▄                                                       
  ▁▁▂▄█████▅▃▃▃▄▅▅▆▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  231 μs           Histogram: frequency by time          265 μs <

 Memory estimate: 182.20 KiB, allocs estimate: 2980.

In [26]:
@benchmark d_dt(xi, neq(log(Sum("j", alpha * xi  * yj + Sum("k", beta * gamma_j * gamma_k * xi * xj * xk)))))

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  171.250 μs …  11.823 ms  ┊ GC (min … max): 0.00% … 98.27%
 Time  (median):     175.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   183.952 μs ± 220.523 μs  ┊ GC (mean ± σ):  3.98% ±  3.74%

        ▃▅▆█▆▄▃                                                  
  ▁▁▂▃▄▇███████▇▆▅▄▄▄▄▄▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  171 μs           Histogram: frequency by time          190 μs <

 Memory estimate: 140.23 KiB, allocs estimate: 2249.

In [27]:
# check if all abstracts are gone
function contains_abstract(term::qExpr)::Bool
    return any([contains_abstract(t) for t in term.terms])
end
function contains_abstract(term::T)::Bool where T<:qComposite
    return contains_abstract(term.expr)
end
function contains_abstract(term::T)::Bool where T<:qMultiComposite
    return any([contains_abstract(t) for t in term.expr])
end
function contains_abstract(term::qAtomProduct)::Bool
    return any([isa(t, qAbstract) for t in term.expr])
end
function contains_abstract(term::diff_qEQ)::Bool 
    return contains_abstract(term.expr) && contains_abstract(term.left_hand_side)
end
# Test 
println("Before: ", contains_abstract(expr)) 
println("After:  ", contains_abstract(substitute(A, xi, dA_dt)))

Before: true
After:  false


In [ ]:
Is = Union{Int,Vector{Int}}
function vecvec_or(A::Vector{Vector{Bool}}, B::Vector{Vector{Bool}})
    # Assume they are equally shaped. 
    return broadcast.(|, A, B)
end
cnimp(a::Bool, b::Bool) = b && !a
function converse_nonimplication(A::Vector{Vector{Bool}}, B::Vector{Vector{Bool}})
    # Assume they are equally shaped. 
    return broadcast.(cnimp, A, B) 
end

# FFunctions
function which_continuum_acting(f::FAtom, where_continuums_f::Vector{Vector{Vector{Int}}})::Vector{Vector{Bool}}
    where_non_trivial::Vector{Vector{Bool}} = []
    for where_f in where_continuums_f
        push!(where_non_trivial, reduce(.|, [f.var_exponents[w] .!= 0 for w in where_f]))
    end
    return where_non_trivial
end
function which_continuum_acting(f::FSum, where_continuums_f::Vector{Vector{Vector{Int}}})::Vector{Vector{Bool}}
    # or of the individual terms 
    return reduce(vecvec_or, [which_continuum_acting(t, where_continuums_f) for t in f.terms])
end
function which_continuum_acting(f::FRational, where_continuums_f::Vector{Vector{Vector{Int}}})::Vector{Vector{Bool}}
    return vecvec_or(which_continuum_acting(f.numer, where_continuums_f), which_continuum_acting(f.denom, where_continuums_f))
end

# QObjs
function which_continuum_acting(q::qAtom, continuum_indexes::Vector{Vector{Int}}, neutral_continuums_op::Vector{Vector{Is}})::Vector{Vector{Bool}}
    my_continuums::Vector{Vector{Bool}} = []
    for (inds, neutral) in zip(continuum_indexes, neutral_continuums_op)
        push!(my_continuums, q.op_indices[inds] .!= neutral)
    end
    return my_continuums
end
function which_continuum_acting(q::qAbstract, continuum_indexes::Vector{Vector{Int}}, neutral_continuums_op::Vector{Vector{Is}})
    error("Which continuum acting should be applied to abstractless expressions!")
end
function which_continuum_acting(q::qAtomProduct)::Vector{Vector{Bool}}
    # xor between vectors of vector of bool 
    qspace = q.statespace
    return vecvec_or(reduce(vecvec_or, [which_continuum_acting(t, qspace.continuum_indexes, qspace.neutral_continuum_op) for t in q.expr]), which_continuum_acting(q.coeff_fun, qspace.where_by_continuum_var))
end

which_continuum_acting (generic function with 6 methods)

In [29]:
function are_indexes_defined(q::qAtomProduct, where_defined::Vector{Vector{Bool}})::Bool
    # check that no true on which_continuum_acting, that isn't also a true on where_defined => converse nonimplication cnimp(a::Bool, b::Bool) = b && !a
    qspace = q.statespace
    if any(any, converse_nonimplication(where_defined, which_continuum_acting(q)))
        x = converse_nonimplication(where_defined, which_continuum_acting(q))
        error("Cannot use an undefined continuums-index on the right hand side of a differential equation! $x")
    end
    return true
end
function are_indexes_defined(q::qExpr, where_defined::Vector{Vector{Bool}})::Bool
    # check element wise if all indexes are defined
    return all([are_indexes_defined(t, where_defined) for t in q.terms])
end
function are_indexes_defined(q::T, where_defined::Vector{Vector{Bool}})::Bool where T <:qComposite
    return are_indexes_defined(q.expr, where_defined)
end
function are_indexes_defined(q::T, where_defined::Vector{Vector{Bool}})::Bool where T <:qMultiComposite
    return all([are_indexes_defined(t, where_defined) for t in q.exprs])
end
function are_indexes_defined(q::qSum, where_defined::Vector{Vector{Bool}})::Bool
    # add the qSum summation indexes 
    subsystem = q.subsystem_index 
    element_indexes = q.element_indexes
    # check where subsystem is among qspace.where_continuum
    outer_ind = findfirst(x -> x == subsystem, q.statespace.where_continuum)
    if outer_ind == nothing
        error("Subsystem $subsystem not found in qspace.subspace_by_ind!")
    end
    if !all(where_defined[outer_ind][element_indexes] .== false)
        error("Summation indexes already defined, cannot sum over defined indexes!")
    end
    new_where_defined = deepcopy(where_defined)
    new_where_defined[outer_ind][element_indexes] .= true
    # check the summation qExpr 
    return are_indexes_defined(q.expr, new_where_defined)
end
function are_indexes_defined(q::diff_qEQ)::Bool
    qspace = q.statespace
    # first two arguments for operators 
    # final argument for paramete/variables
    defined = which_continuum_acting(q.left_hand_side)
    # check recursively if there are undefined elements in the right hand side
    return are_indexes_defined(q.expr, defined)
end
# Test
are_indexes_defined(dx_dt)

true

In [30]:
function where_defined_to_index_order(statespace::StateSpace, where_defined::Vector{Vector{Bool}})::Tuple{Vector{Int}, Vector{Int}}
    # takes where_defined and the continuum indexes to determine the new order for both operators and variables 
    # for each element of where_defined, we shift all the true elements to the left, and all false elements to the right, we want to get the indexes of the permutation that achieves that 
    n_vars = length(statespace.vars_str)
    n_ops = length(statespace.neutral_op)
    op_inds = collect(1:n_ops)
    var_inds = collect(1:n_vars)
    continuum_indexes = qspace.continuum_indexes
    variable_indexes = qspace.where_by_continuum_var
    for (w, c, vs)  in zip(where_defined, continuum_indexes, variable_indexes)
        w_order = sortperm(w, rev=true)
        for v in vs
            var_inds[v] = var_inds[v][w_order]
        end
        op_inds[c] = op_inds[c][w_order]
    end
    return op_inds, var_inds
end
### FFunction
function reorder(f::FAtom, var_index_order::Vector{Int})::FAtom
    var_exponents = f.var_exponents[var_index_order]
    return FAtom(copy(f.coeff), var_exponents)
end
function reorder(f::FSum, var_index_order::Vector{Int})::FSum
    f.terms = [reorder(ff, var_index_order) for ff in f.terms]
    return f
end
function reorder(f::FRational, var_index_order::Vector{Int})::FRational
    f.numer = reorder(f.numer, var_index_order)
    f.denom = reorder(f.denom, var_index_order)
    return f
end

# qObj
function reorder(q::qTerm, index_order::Vector{Int})::qTerm
    op_indices = q.op_indices[index_order]
    return qTerm(op_indices)
end
function reorder(q::qAtomProduct, add_at_sum::Bool,  where_defined::Vector{Vector{Bool}}, index_order::Vector{Int}, var_index_order::Vector{Int})::qAtomProduct
    q.expr = [reorder(qq, index_order) for qq in q.expr]
    q.coeff_fun = reorder(q.coeff_fun, var_index_order)
    return q
end
function reorder(q::qExpr, add_at_sum::Bool, where_defined::Vector{Vector{Bool}}, index_order::Vector{Int}, var_index_order::Vector{Int})::qExpr
    q.terms = [reorder(qq, add_at_sum, where_defined, index_order, var_index_order) for qq in q.terms]
    return q
end
function reorder(q::T, add_at_sum::Bool, where_defined::Vector{Vector{Bool}}, index_order::Vector{Int}, var_index_order::Vector{Int})::T where T <: qComposite
    q.expr = reorder(q.expr, add_at_sum, where_defined, index_order, var_index_order)
    return q
end
function reorder(q::T, add_at_sum::Bool, where_defined::Vector{Vector{Bool}}, index_order::Vector{Int}, var_index_order::Vector{Int})::T where T <: qMultiComposite
    q.expr = [reorder(qq, add_at_sum, where_defined, index_order, var_index_order) for qq in q.expr]
    return q
end
function reorder(q::qSum, add_at_sum::Bool, where_defined::Vector{Vector{Bool}}, index_order::Vector{Int}, var_index_order::Vector{Int})::qSum
    # define improved index_order and var_index_order
    qspace = q.statespace
    if add_at_sum
        subsystem = q.subsystem_index 
        element_indexes = q.element_indexes
        # check where subsystem is among qspace.where_continuum
        outer_ind = findfirst(x -> x == subsystem, q.statespace.where_continuum)
        if outer_ind == nothing
            error("Subsystem $subsystem not found in qspace.subspace_by_ind!")
        end
        if !all(where_defined[outer_ind][element_indexes] .== false)
            error("Summation indexes already defined, cannot sum over defined indexes!")
        end
        new_where_defined = deepcopy(where_defined)
        new_where_defined[outer_ind][element_indexes] .= true
        op_ind, var_inds = where_defined_to_index_order(q.statespace, new_where_defined)

        # we need to change the other parameters of sum aswell determining what is summed over 
        #qspace.continuum_indexes, qspace.neutral_continuum_op 
        curr_subspace = qspace.subspaces[subsystem]
        new_element_indexes::Vector{Int} = []
        new_indexes::Vector{String} = []
        for (i, element_index) in enumerate(element_indexes)
            prev_index = curr_subspace.op_index_inds[element_index]
            new_ind = findfirst(x -> x == prev_index, op_ind)
            if new_ind == nothing
                error("Not all element_indexes found in new indexing!")
            end
            # find in subspace 
            new_subind = findfirst(x -> x == new_ind, curr_subspace.op_index_inds)
            if new_subind == nothing
                error("Not all element_indexes found in new indexing!")
            end
            push!(new_element_indexes, new_subind)
            push!(new_indexes, curr_subspace.keys[new_subind])
        end
        q.expr = reorder(q.expr, add_at_sum, new_where_defined, op_ind, var_inds)
        q.indexes = new_indexes
        q.element_indexes = new_element_indexes
    else
        q.expr = reorder(q.expr, add_at_sum, where_defined, index_order, var_index_order)
    end
    return q
end
function reorder!(q::diff_qEQ)::diff_qEQ
    # check index order on left side 
    q = copy(q)#deepcopy(q)
    where_defined_lhs = which_continuum_acting(q.left_hand_side)
    op_inds, var_inds = where_defined_to_index_order(q.statespace, where_defined_lhs)
    # check if op_inds is not sorted (i.e. not equal to 1:length(op_inds))
    if op_inds != 1:length(op_inds) 
        # first we need to sort without changing at sums 
        left_hand_side = reorder(q.left_hand_side, false, where_defined_lhs, op_inds, var_inds)
        # then expr 
        expr = reorder(q.expr, false, where_defined_lhs, op_inds, var_inds)
        where_defined_lhs = which_continuum_acting(left_hand_side)
        op_inds = collect(1:length(op_inds))
        var_inds = collect(1:length(var_inds))
        q = diff_qEQ(q.statespace, left_hand_side, expr, q.braket)
    end
    expr = reorder(q.expr, true, where_defined_lhs, op_inds, var_inds)
    return diff_qEQ(q.statespace, q.left_hand_side, expr, q.braket)
end
# Test 
expr = neq(log(Sum("j", alpha * A * yj + Sum("k", beta * gamma_j * gamma_k * A * xj * xk))))
dA_dt = d_dt(A, expr)
dx_dt = substitute(A, xi, dA_dt)
reorder!(dx_dt)


d(σˣᵢ) / dt = log(-iασᶻᵢ+2β(t)γᵢ²σˣᵢ+∑ⱼ(ασˣᵢσʸⱼ+β(t)γⱼ²σˣᵢ+2β(t)γᵢγⱼσˣⱼ)+∑₍ⱼₖ₎β(t)γⱼγₖσˣᵢσˣⱼσˣₖ)

In [11]:
@benchmark reorder!($dx_dt)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  14.083 μs …  10.159 ms  ┊ GC (min … max): 0.00% … 99.39%
 Time  (median):     14.916 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.309 μs ± 101.449 μs  ┊ GC (mean ± σ):  6.19% ±  0.99%

     ▃▄▇█▅▂                                                     
  ▁▂▆██████▅▆▅▄▃▂▂▂▁▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  14.1 μs         Histogram: frequency by time         21.1 μs <

 Memory estimate: 29.48 KiB, allocs estimate: 606.

In [ ]:
reorder!(dx_dt)

d(σˣᵢ) / dt = log(-iασᶻᵢ+2β(t)γᵢ²σˣᵢ+∑ⱼ(ασˣᵢσʸⱼ+β(t)γⱼ²σˣᵢ+2β(t)γᵢγⱼσˣⱼ)+∑₍ⱼₖ₎β(t)γⱼγₖσˣᵢσˣⱼσˣₖ)

In [36]:
function where_acting_index(q::qTerm, statespace::StateSpace)::Vector{Int}
    return [i for (i, op) in enumerate(q.op_indices) if op!=statespace.neutral_op[i]]
end
# Order of qTerm 
function order(q::qTerm, statespace::StateSpace)::Int
    # Determines the order of a qTerm operator 
    return sum(where_acting(q, statespace))
end
# Test 
q = (xi*b*yj).terms[1].expr[1]
println(where_acting_index(q, qspace))
println(order(q, qspace))

[2, 3, 5]
3


In [1]:
# Define necessary Structs
struct IndexedProduct
    coeff::Int 
    indices::Vector{Vector{Int}}
end
struct IndexedCumulant
    n::Int 
    cumulant::Vector{IndexedProduct}
end 
import Base:-
function -(a::IndexedProduct)::IndexedProduct 
    return IndexedProduct(-a.coeff, a.indices)
end

# Define Operations to Construct and Raise the Level of the Structs!
function FirstIndexedCumulant(i::Int=1)
    return IndexedCumulant(1, [IndexedProduct(1, [[i]])])
end
function raising_operator(new_op::Int, prev_product::IndexedProduct)::Vector{IndexedProduct}
    # Raising operator is distributve with respect to products
    # new_op | [i1, i2, i3] = [new_op, i1, i2, i3] - [i1, i2, i3] [new_op]
    # new_op | [[A],[B],...] = [new_op | [A], new_op | [B], ...]
    # The sum of applying itto each Vector{Int} in the product
    n = length(prev_product.indices)
    prev_coeff = prev_product.coeff

    new_product::Vector{IndexedProduct} = []
    for i in 1:length(prev_product.indices)
        new_c = [ copy(v) for v in prev_product.indices ]
        push!(new_c[i], new_op)
        push!(new_product, IndexedProduct(prev_coeff, new_c))
    end
    c = [ copy(v) for v in prev_product.indices ]
    push!(c, [new_op])
    push!(new_product, IndexedProduct(-n*prev_coeff, c))
    return new_product 
end

function raising_operator(new_op::Int, prev_cumulant::IndexedCumulant)
    # Raising operator is distributve with respect to products  
    # and linear with respect to sum   
    cumulant = prev_cumulant.cumulant
    new_cumulant = raising_operator(new_op, cumulant[1]) 
    for i in 2:length(cumulant)
        append!(new_cumulant, raising_operator(new_op, cumulant[i]))
    end
    return IndexedCumulant(prev_cumulant.n+1, new_cumulant)
end
function IndexedCumulant(order::Int)
    if order == 0 
        error("Order must be greater than 0")
    end
    cum = FirstIndexedCumulant()
    for i in 2:order
        cum = raising_operator(i, cum)
    end
    return cum
end
# Test 
cum = IndexedCumulant(3)

IndexedCumulant(3, IndexedProduct[IndexedProduct(1, [[1, 2, 3]]), IndexedProduct(-1, [[1, 2], [3]]), IndexedProduct(-1, [[1, 3], [2]]), IndexedProduct(-1, [[1], [2, 3]]), IndexedProduct(2, [[1], [2], [3]])])

In [2]:
struct Reduced_IndexedCumulant
    operator::IndexedProduct
    aproximation::Vector{IndexedProduct}
end
function Reduced_IndexedCumulant(order::Int)
    full_cumulant = IndexedCumulant(order)
    operator = full_cumulant.cumulant[1]
    aproximation = .-full_cumulant.cumulant[2:end]
    return Reduced_IndexedCumulant(operator, aproximation)
end
# Test 
cum = Reduced_IndexedCumulant(3)

Reduced_IndexedCumulant(IndexedProduct(1, [[1, 2, 3]]), IndexedProduct[IndexedProduct(1, [[1, 2], [3]]), IndexedProduct(1, [[1, 3], [2]]), IndexedProduct(1, [[1], [2, 3]]), IndexedProduct(-2, [[1], [2], [3]])])

In [ ]:
struct qCumulant <:qComposite  # Must be qComposite to be in qExpr's
    statespace::StateSpace
    atom::qAtom
    expr_::qExpr
    order::Int
    where_acting::Vector{Int}
end


In [35]:
# Add commulants 

# Add Equation Set
# Add Equation Set to Indexed Equation Set with indexed  

# add index version of qAtomProduct.
# add evaluate 

# add distribution sampling 

# change do_sigma to custom symbols 

In [9]:
simplify(qCommutator(Sum("i", alpha*ph*xi*yi) + zj,zh))

[zⱼ+∑ᵢ⁼iαpₕzᵢ, zₕ]

In [10]:
simplify(exp(Sum("i", alpha*ph*xi*yi) + zj)+zh)

zₕ+exp(zⱼ+∑ᵢ⁼iαpₕzᵢ)

In [11]:
log(Sum("i", alpha*ph*xi*yi) + zj)

log(zⱼ+∑ᵢ⁼iαpₕzᵢ)

In [12]:
simplify(power(Sum("i", alpha*ph*xi*yi) + zj,2)+zh)

zₕ+(zⱼ+∑ᵢ⁼iαpₕzᵢ)²

In [13]:
simplify(root(Sum("i", alpha*ph*xi*yi) + zj,2)+zh)

zₕ+(zⱼ+∑ᵢ⁼iαpₕzᵢ)⁼²